<a href="https://colab.research.google.com/github/Kalima83/procesamiento_lenguaje_natural_Desafios/blob/main/desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Consigna del desafío 2
Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado

Recuerden que su notebook de entrega debe poder correrse de inicio a fin sin la aparición de errores.

1. Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.
2. Elegir términos de interés y buscar términos más similares y menos similares.
3. Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.
4. Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse. Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 40.1 MB/s eta 0:00:00


## Importar dataset Song

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2025-11-13 03:05:39--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-11-13 03:05:39--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-11-13 03:05:40--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

['Kanye_West.txt',
 'lil-wayne.txt',
 'blink-182.txt',
 'eminem.txt',
 'bob-marley.txt',
 'michael-jackson.txt',
 'amy-winehouse.txt',
 'bruce-springsteen.txt',
 'alicia-keys.txt',
 'bob-dylan.txt',
 'ludacris.txt',
 'bruno-mars.txt',
 'rihanna.txt',
 'nursery_rhymes.txt',
 'prince.txt',
 'Lil_Wayne.txt',
 'cake.txt',
 'kanye.txt',
 'dickinson.txt',
 'lin-manuel-miranda.txt',
 'lorde.txt',
 'al-green.txt',
 'joni-mitchell.txt',
 'r-kelly.txt',
 'leonard-cohen.txt',
 'notorious-big.txt',
 'nickelback.txt',
 'adele.txt',
 'patti-smith.txt',
 'bjork.txt',
 'radiohead.txt',
 'janisjoplin.txt',
 'kanye-west.txt',
 'missy-elliott.txt',
 'bieber.txt',
 'britney-spears.txt',
 'jimi-hendrix.txt',
 'johnny-cash.txt',
 'beatles.txt',
 'drake.txt',
 'dr-seuss.txt',
 'dj-khaled.txt',
 'nirvana.txt',
 'notorious_big.txt',
 'paul-simon.txt',
 'dolly-parton.txt',
 'nicki-minaj.txt',
 'lady-gaga.txt',
 'disney.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/nirvana.txt', sep='/n', header=None)
df.head()

/tmp/ipython-input-1516256778.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/nirvana.txt', sep='/n', header=None)


,0
0,"Load up on guns, bring your friends"
1,It's fun to lose and to pretend
2,She's over-bored and self-assured
3,"Oh no, I know a dirty word Hello, hello, hello..."
4,"Hello, hello, hello, how low"


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1985


### 1. Preprocesamiento

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))
sentence_tokens[:2]

[['load', 'up', 'on', 'guns', 'bring', 'your', 'friends'],
 ["it's", 'fun', 'to', 'lose', 'and', 'to', 'pretend']]

### 2. Crear Vector

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback para imprimir la pérdida después de cada época
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 1985
Cantidad de words distintas en el corpus: 381


In [ ]:
terms_to_check = ['guns', 'friends', 'lose']

print("Checking terms in model vocabulary:")
for term in terms_to_check:
    if term in w2v_model.wv.key_to_index:
        print(f"  '{term}' is present in the vocabulary.")
    else:
        print(f"  '{term}' is NOT present in the vocabulary.")

Checking terms in model vocabulary:
  'guns' is NOT present in the vocabulary.
  'friends' is present in the vocabulary.
  'lose' is present in the vocabulary.


In [ ]:
terms_of_interest = ['friends', 'lose']

print("\nMost similar terms:")
for term in terms_of_interest:
    if term in w2v_model.wv.key_to_index:
        print(f"\nTerms similar to '{term}':")
        for word, score in w2v_model.wv.most_similar(term):
            print(f"  {word}: {score:.4f}")
    else:
        print(f"  '{term}' not in vocabulary, skipping similar terms.")


Most similar terms:

Terms similar to 'friends':
  for: 0.1565
  return: 0.1555
  the: 0.1516
  out: 0.1460
  house: 0.1458
  pretend: 0.1411
  full: 0.1383
  surely: 0.1279
  your: 0.1262
  sunday: 0.1239

Terms similar to 'lose':
  all: 0.1942
  jar: 0.1644
  there: 0.1328
  sad: 0.1271
  free: 0.1221
  shine: 0.1172
  fear: 0.1169
  and: 0.1142
  take: 0.1140
  travel: 0.1130


In [ ]:
groups_for_doesnt_match = [
    ['guns', 'friends', 'fire', 'bullet'],
    ['friends', 'enemies', 'family', 'people'],
    ['lose', 'win', 'pretend', 'gain']
]

print("\nLeast matching terms:")
for i, group in enumerate(groups_for_doesnt_match):
    valid_words = [word for word in group if word in w2v_model.wv.key_to_index]

    if len(valid_words) < 2:
        print(f"\nSkipping group {i+1} ({', '.join(group)}) because not enough words are in the vocabulary after filtering. (Valid words: {', '.join(valid_words) if valid_words else 'None'})")
    else:
        print(f"\nFor group {i+1} ({', '.join(group)}), considering words: ({', '.join(valid_words)}):")
        try:
            doesnt_match_term = w2v_model.wv.doesnt_match(valid_words)
            print(f"  The term that doesn't match is: '{doesnt_match_term}'")
        except Exception as e:
            print(f"  Could not perform doesnt_match for this group. Error: {e}")


Least matching terms:

Skipping group 1 (guns, friends, fire, bullet) because not enough words are in the vocabulary after filtering. (Valid words: friends)

For group 2 (friends, enemies, family, people), considering words: (friends, people):
  The term that doesn't match is: 'friends'

For group 3 (lose, win, pretend, gain), considering words: (lose, pretend):
  The term that doesn't match is: 'lose'


## Summary:

### Data Analysis Key Findings

*   **Vocabulary Presence:** Out of the initial terms of interest, 'guns' was not present in the Word2Vec model's vocabulary, while 'friends' and 'lose' were available for analysis.
*   **Similar Terms to 'friends':** The most similar terms included 'for', 'return', 'the', 'out', 'house', 'pretend', 'full', 'surely', 'your', and 'sunday'. These largely functional or contextual words suggest the model captures co-occurrence in Nirvana's lyrics rather than deep semantic kinship.
*   **Similar Terms to 'lose':** For 'lose', the top similar terms were 'all', 'jar', 'there', 'sad', 'free', 'shine', 'fear', 'and', 'take', and 'travel'. Notably, 'sad' and 'fear' align semantically with the concept of 'lose', while others are functional or context-specific.
*   **Least Matching Term for Group 1 (['guns', 'friends', 'fire', 'bullet']):** This group analysis was largely ineffective as 'guns', 'fire', and 'bullet' were not in the vocabulary, leaving only 'friends' for consideration.
*   **Least Matching Term for Group 2 (['friends', 'enemies', 'family', 'people']):** When analyzed with the available terms `['friends', 'people']`, the model identified **'friends'** as the term that didn't match, suggesting a distinct usage pattern for 'friends' in the corpus compared to 'people'.
*   **Least Matching Term for Group 3 (['lose', 'win', 'pretend', 'gain']):** For the valid terms `['lose', 'pretend']`, the model found **'lose'** to be the least matching term, indicating a subtle semantic difference between 'lose' and 'pretend' within the specific lyrical context.

### Insights or Next Steps

*   The corpus size and specific lyrical nature of Nirvana's songs significantly influence word embeddings, leading to co-occurrence-based similarities for some terms rather than broad semantic ones.
*   To gain a more robust understanding of term relationships, it would be beneficial to train the Word2Vec model on a larger and more diverse lyrical corpus or to use a pre-trained model on a general language dataset for comparison.
